<a href="https://colab.research.google.com/github/Loujainl/word_guessing/blob/main/word_guessing_game_tiime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Tiime scientific coding test


*Alice* and Bob play the following game:
Alice chooses a 10-letter word from a fixed dictionary, uniformly at random
At each turn Bob proposes a word from the same dictionary
For each proposal, Alice returns the "score" of the proposed word: each correct letter gives 1 point, plus an additional point if it is at the right position.
The total score of Bob is the sum of scores over k rounds with k fixed to 5, 10 or 15.

The goal is to maximize the score of Bob over 100 games - where each game lasts k rounds - (or its expectation). The score is then normalized so that the maximum score is 100.

You are allowed to use reasonable precomputation (less than one hour) and your code should play any game in a reasonable amount of time (less than a few minutes). Running time will also be considered in grading. You can use any library, even if it is not installed. To install a library with pip, just paste the command with ! before: !pip install numpy (this one is already installed).

Create a copy of this notebook that you will share. Runtime > Run all should execute your code without error. If you have suggestions for improvements, feel free to add them and we might take them into account in the evaluation.

Typical good scores we expect are for instance: 30 (k=5), 60 (k=10), 74 (k=15).

Please send your score for each size along with the link to your solution notebook.

##Testing Code

Execute the code below to download the word list and define the testing class. It is recommended to read this code.

**DO NOT MODIFY ANYTHING IN THIS SECTION**

In [ ]:
!wget -nc https://gematria.tech/words_10.txt 2> /dev/null

In [ ]:
from collections import Counter

def score(word1, word2):
    """scoring function
    """
    assert len(word1) == len(word2)
    ans = 0
    hist = Counter(word1)
    for c in word2:
        if hist[c]:
            hist[c] -= 1
            ans += 1
    for c1, c2 in zip(word1, word2):
        ans += c1 == c2
    return ans

assert score("abc", "abd") == 4
assert score("abc", "bca") == 3

In [ ]:
import random
from tqdm.autonotebook import tqdm

class Simulator:
    def __init__(self, path="words_10.txt"):
        with open(path) as f:
            self.words = list(map(str.strip, f))
            self.words_set = self.words

    def play(self, bob, rounds):
        total_score = 0
        word = random.choice(self.words)
        bob.new_game()
        for _ in range(rounds):
            guess = bob.guess()
            if guess not in self.words_set:
                return 0
            s = score(word, guess)
            bob.clue(s)
            total_score += s
        return total_score

    def score(self, bob, games, rounds):
        ans = sum(self.play(bob, rounds) / 20 / rounds for _ in tqdm(range(games)))
        return 10000 * ans // 100 / games


##My Code

In [ ]:
from collections import defaultdict, Counter

class Bob:
    """
    This class represents Bob, the player in the game.
    """

    def __init__(self, words, rounds):
        """
        Initializes the Bob class with a list of words and the number of rounds.
        Precomputes various letter frequencies.
        """
        self.words = words
        self.rounds = rounds
        self.word_length = len(words[0])
        self.letter_freq = self.compute_letter_frequencies()
        self.following_letter_freq = self.compute_following_letter_frequencies()
        self.positional_letter_freq = self.compute_positional_letter_frequencies()
        self.candidate_indices = set(range(len(words)))
        self.last_guess_index = None

    def compute_letter_frequencies(self):
        """
        Computes the frequency of each letter in the entire word list.
        """
        letter_freq = Counter()
        for word in self.words:
            letter_freq.update(word)
        return letter_freq

    def compute_following_letter_frequencies(self):
        """
        Computes the frequency of each letter following every other letter in the word list.
        """
        following_letter_freq = defaultdict(Counter)
        for word in self.words:
            for i in range(len(word) - 1):
                following_letter_freq[word[i]][word[i + 1]] += 1
        return following_letter_freq

    def compute_positional_letter_frequencies(self):
        """
        Computes the frequency of each letter at each position in the words.
        """
        positional_letter_freq = [Counter() for _ in range(self.word_length)]
        for word in self.words:
            for i, letter in enumerate(word):
                positional_letter_freq[i][letter] += 1
        return positional_letter_freq

    def new_game(self):
        """
        Prepares for a new game by resetting the candidate indices and the last guess index.
        """
        self.candidate_indices = set(range(len(self.words)))
        self.last_guess_index = None

    def guess(self):
        """
        Makes a new guess. If it's the first guess, makes a greedy guess based on letter frequencies.
        Otherwise, chooses the best guess based on remaining candidates.
        """
        self.last_guess_index = self.choose_best_guess()
        return self.words[self.last_guess_index]

    def make_greedy_guess(self):
        """
        Makes an initial guess based on the sum of positional letter frequencies.
        """
        candidate_scores = []
        for i in self.candidate_indices:
            word = self.words[i]
            score = sum(self.positional_letter_freq[j][word[j]] for j in range(self.word_length))
            candidate_scores.append((score, i))
        candidate_scores.sort(reverse=True)
        self.last_guess_index = candidate_scores[0][1]
        return self.last_guess_index

    def clue(self, score):
        """
        The last guess got this score.
        Updates the candidate indices based on the score of the last guess.
        """
        self.candidate_indices = self.update_candidates(score)

    def choose_best_guess(self):
        """
        Chooses the best guess from the remaining candidates randomly.
        """
        return random.choice(list(self.candidate_indices))

    def update_candidates(self, score):
        """
        Filters the candidate indices to only include words that would give the same score
        when compared to the last guessed word.
        """
        new_candidates = set()
        for idx in self.candidate_indices:
            if self.compute_score(self.words[self.last_guess_index], self.words[idx]) == score:
                new_candidates.add(idx)
        return new_candidates

    def compute_score(self, word1, word2):
        """
        Computes the score between two words. The score is the sum of common letters and exact matches.
        """
        common = sum((Counter(word1) & Counter(word2)).values())
        exact = sum(a == b for a, b in zip(word1, word2))
        return common + exact

##Evaluation

In [ ]:
from time import time

simulator = Simulator()

for rounds in [5, 10, 15]:
    t = time()
    bob = Bob(simulator.words.copy(), rounds)
    print('%.02f s' % (time() - t))
    print("score", simulator.score(bob, games=100, rounds=rounds))


0.30 s


  0%|          | 0/100 [00:00<?, ?it/s]

score 29.74
0.62 s


  0%|          | 0/100 [00:00<?, ?it/s]

score 61.06
0.32 s


  0%|          | 0/100 [00:00<?, ?it/s]

score 74.12
